In [1]:
from selenium import webdriver  # driver
from selenium.webdriver.chrome.options import Options  # setting User Agent
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException, NoSuchWindowException
from selenium.webdriver.common.by import By # helps implicit waiting
from selenium.webdriver.support import expected_conditions as EC # this one too
from selenium.webdriver.support.ui import WebDriverWait # implicit waiting
from selenium.webdriver import ActionChains
import time
import csv
from ipywidgets import IntProgress
from IPython.display import display

In [2]:
data = open('input.csv', encoding='utf-8')  # opens the input file you shared
csv_data = csv.reader(data)
doctor_urls = list(csv_data)  # then, it transforms the data into a list of list
all_urls = []
# here, it unpacks the list of lists
# and sorts it into one list, namely all_urls
for urls in doctor_urls:
    for url in urls:
        all_urls.append(url)
data.close()

In [3]:
path = "C:\webdrivers\chromedriver_win32(104)\chromedriver.exe"
agent = "user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36"
headless = "--headless"
the_options = webdriver.ChromeOptions()
the_options.add_argument(path)
the_options.binary_location = "C:\Program Files\Google\Chrome Beta\Application\chrome.exe"
the_options.add_argument(headless)
driver = webdriver.Chrome(executable_path=path, options=the_options)
driver.maximize_window()

In [4]:
headers = ['URL', 'Name', 'About',
   'Special Interests', 'Practice Location','Address',
   'Telephone', 'Hospital Affiliations'] # first row of any good CSV file are column headers
table = []
table.append(headers)

max_count = len(all_urls)
bar = IntProgress(min=0, max=max_count, description='Starting...')
display(bar)

for url in all_urls:
    driver.get(url)
    driver.implicitly_wait(5)
    bar.value +=1
    bar.description = f"[{bar.value} of {max_count}]"
    locations = WebDriverWait(driver, 1).until(
        EC.presence_of_all_elements_located((By.XPATH, "//li[starts-with(@class, 'location')]"))
    )
    for location in locations:
        index = locations.index(location)
        if index == 0:
            url = url
            name = driver.find_element_by_xpath("//header[@class='card-profile']/div/div/h1").text
            about = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.XPATH, "//article[@class='bio']/div[1]"))
            ).text
            try:
                special_interests = driver.find_element_by_xpath("//article[@class='bio']/div[2]/div").text
            except NoSuchElementException:
                special_interests = ''
            practice_location = driver.find_element_by_xpath("//li[starts-with(@class, 'location')][1]/*/h3").text
            address = driver.find_element_by_xpath("//li[starts-with(@class, 'location')][1]/*/address").text
            phone_button = WebDriverWait(driver, 3).until(
                EC.visibility_of_element_located((By.XPATH, "(//button[contains(@title, 'Click to show phone number')])[1]"))
            )
            driver.execute_script("arguments[0].click();", phone_button)
            time.sleep(1)
            phone_number = driver.find_element_by_xpath("//b/a").text
            time.sleep(1)
            close_button = driver.find_element_by_xpath("//button[@class='dismiss']").click()
            try:
                hospital_affiliations = [affiliation.text for affiliation in driver.find_elements_by_xpath("//*[contains(text(), 'Hospital affiliations')]/../ul/li")]
                hospital_affiliations = '\n'.join(hospital_affiliations)
            except NoSuchElementException:
                pass

            first_record = [url, name, about,
                           special_interests, practice_location, address,
                           phone_number, hospital_affiliations]

            table.append(first_record)

        elif index != 0:
            url = ''
            name = ''
            about = ''
            special_interests = ''
            practice_location = driver.find_element_by_xpath(f"//li[starts-with(@class, 'location')][{index + 1}]/*/h3").text
            address = driver.find_element_by_xpath(f"//li[starts-with(@class, 'location')][{index + 1}]/*/address").text
            phone_button = WebDriverWait(driver, 3).until(
                EC.visibility_of_element_located((By.XPATH, "(//button[contains(@title, 'Click to show phone number')])[1]"))
            )
            driver.execute_script("arguments[0].click();", phone_button)           
            time.sleep(1)
            phone_number = driver.find_element_by_xpath("//b/a").text
            time.sleep(1)
            close_button = driver.find_element_by_xpath("//button[@class='dismiss']").click()
            hospital_affiliations = ''
            sub_record = [url, name, about,
                          special_interests, practice_location, address,
                          phone_number, hospital_affiliations]

            table.append(sub_record)

IntProgress(value=0, description='Starting...', max=200)

In [5]:
csv_file = open('output.csv','w',newline='')
csv_writer = csv.writer(csv_file,delimiter=',')
csv_writer.writerows(table)
csv_file.close()